In [1]:
import os
import sys
sys.path.append(os.path.join('..'))

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
%load_ext autoreload
%autoreload 2

from llms_content import content

## Init the model. We will start with the api solution

In [3]:
openai_model = "gpt-3.5-turbo-0125" # "gpt-4o" "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0125"

# Count tokens with Tiktoken
https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken

In [12]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(content, openai_model)

2315

# Generate a single role-play card.

1. Set a project name.
2. They were suggesting me a stream there, but i am not sure if i can use it in telegram. <br> https://cookbook.openai.com/examples/how_to_stream_completions

In [5]:
def format_json_pretty(json_data):
    formatted_str = ""
    for key, value in json_data.items():
        if isinstance(value, dict):
            formatted_str += f"**{key}**\n"
            for sub_key, sub_value in value.items():
                formatted_str += f"  {sub_key}. {sub_value}\n"
        elif isinstance(value, list):
            formatted_str += f"**{key}**\n"
            for sub_value in value:
                formatted_str += f"  - {sub_value}\n"
        else:
            formatted_str += f"**{key}**: {value}\n"
    return formatted_str

In [6]:
import json
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('OPENAI_API_TOKEN'),
    organization=os.getenv('OPENAI_ORG_ID'), # unnecessary
    project=os.getenv('OPENAI_PROJ_OET'), # to track expenses
)

In [7]:
print(content)


You are a medicine spiking card for the OET exam creating AI tool. Don't so anything except cards creation. 
Here is an example of a card: 
{
    Patient Card: {
        SETTING: Local Medical Clinic
    
        PATIENT: You are 45 years old and recovering from a mild heart attack which you had two weeks ago. You were discharged from hospital four days ago. You are seeing the doctor because you are unsure how much physical activity is appropriate; you’re concerned it might bring on another heart attack.
        
        PATIENT TASK:
        - When asked, say you’ve felt much weaker and very tired since the heart attack; you’re worried that any physical activity, such as walking, gardening or swimming, might bring on another heart attack.
        - Say you’d like to know how much physical activity is advisable.
        - When asked, say you’re an office worker. You sit at a desk all day; you don’t get much exercise at work.
        - When asked, say you’re worried about having anothe

In [8]:
%%time

completion = client.chat.completions.create(
    model=openai_model,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": content},
        {"role": "user", "content": "Create an OET card in JSON format"}
    ],
    # stream=True, # don't need for card generation
    max_tokens=1000
)

card_content = completion.choices[0].message.content
card_json = json.loads(card_content)

# print(completion.model, completion.usage.to_dict())

assert len(card_json.items()) == 2, 'The card should consist of two parts'

cards = {}
for card_name, (key, val) in zip(['patent', 'doctor'], card_json.items()):
    print(key, format_json_pretty(val), sep=':\n')
    cards[card_name] = (key + ':\n' + format_json_pretty(val))

Patient Card:
**SETTING**: Dermatology Clinic
**PATIENT**: You are a 30-year-old office worker who has been experiencing persistent acne breakouts on your face for the past few months. You have tried over-the-counter products without much success and are feeling self-conscious about your skin. You have come to the dermatology clinic seeking help and advice.
**PATIENT TASKS**
  - Describe to the doctor your concerns about the acne breakouts on your face and how it is affecting your self-esteem.
  - Mention that you have tried over-the-counter products but they haven't been effective in clearing up your acne.
  - Ask the doctor about the possible causes of your acne breakouts and if there are any triggers that may be worsening the condition (e.g., diet, skincare products).
  - Inquire about the treatment options available for acne and ask about potential side effects of the medications.
  - Express your desire to improve your skin condition and ask for recommendations on skincare routine

In [9]:
print(cards['doctor'])

Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: You are a dermatologist seeing a 30-year-old patient who has been struggling with persistent acne breakouts on the face. The patient is feeling self-conscious and seeking help and advice to manage the condition.
**DOCTOR TASKS**
  - Inquire about the duration and severity of the acne breakouts and any previous treatments tried by the patient.
  - Discuss possible causes of acne breakouts including hormonal imbalance, diet, skincare products, and stress (possible causes: hormonal changes, genetic predisposition, diet high in sugar and dairy, use of comedogenic skincare products).
  - Recommend a suitable treatment plan based on the patient's skin condition, which may include topical medications, oral medications, or procedures like chemical peels or laser therapy.
  - Explain the importance of consistent skincare routine, gentle cleansing, non-comedogenic products, and dietary adjustments to help manage acne in the long term.
  - 

# 1. Creating and Sending the Card to the User

In [9]:
def generate_card():
    completion = client.chat.completions.create(
        model=openai_model,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": "Create an OET card in JSON format"}
        ],
        # stream=True, # don't need for card generation
        max_tokens=1000
    )
    
    card_content = completion.choices[0].message.content
    card_json = json.loads(card_content)
    
    # print(completion.model, completion.usage.to_dict())
    
    assert len(card_json.items()) == 2, 'The card should consist of two parts'

    cards = {}
    for card_name, (key, val) in zip(['patient', 'doctor'], card_json.items()):
        cards[card_name] = (key + ':\n' + format_json_pretty(val))
    
    return cards, card_json

formatted_cards, card_json = generate_card()
print(formatted_cards['patient'], formatted_cards['doctor'], sep='\n')

Patient Card:
**SETTING**: Dermatology Clinic
**PATIENT**: You are a 30-year-old woman who has been experiencing severe itching and redness on your skin for the past few weeks. You have noticed the symptoms mainly on your arms, legs, and back. The itching is disrupting your sleep and daily activities, and you have tried over-the-counter creams with no relief.
**PATIENT TASKS**
  - Describe the severity of the itching and redness on your skin, mentioning the areas affected.
  - Express how the itching is impacting your sleep and daily life.
  - Inform the doctor about the unsuccessful use of over-the-counter creams.
  - Ask the doctor about the possible causes of your symptoms and if it could be a skin allergy.
  - Acknowledge the doctor's recommendations and ask about the next steps.

Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily a

## 1.2 Lets try few-shot generation:

In [116]:
import json


oet_cards = [
    {
        "question": "Generate a card for speaking role play in OET exam. The card must be in JSON format with 5 keys: SETTINGS; PATIENT_CARD; PATIENT_TASK; DOCTOR_CARD; DOCTOR_TASK. The disease is heart",
        "answer": """{
            SETTINGS : Local Medical Clinic,
            PATIENT_CARD : You are 45 years old and recovering from a mild heart attack which you had two weeks ago. You were discharged from hospital four days ago. You are seeing the doctor because you are unsure how much physical activity is appropriate; you’re concerned it might bring on another heart attack.,
            PATIENT_TASK : [When asked, say you’ve felt much weaker and very tired since the heart attack; you’re worried that any physical activity, such as walking, gardening or swimming, might bring on another heart attack., Say you’d like to know how much physical activity is advisable., When asked, say you’re an office worker. You sit at a desk all day; you don’t get much exercise at work., When asked, say you’re worried about having another heart attack so you’d like to know what you can do to reduce the risk of further attacks., Say that information has been very helpful. You’ll try to gradually do more exercise and you’ll look into the rehabilitation programme.],
            DOCTOR_CARD : You see a 45-year-old patient who suffered a mild anterior acute myocardial infarction two weeks ago. Recovery was uncomplicated and the patient was discharged from hospital four days ago. He/she is now concerned about how much physical activity is appropriate during recovery,
            DOCTOR_TASK : [Find out reason for visit.", "Reassure patient about fatigue (e.g., expected: 4–6 weeks for return of full energy levels, etc.). Remind patient about heart attack recovery (e.g., gradual, lifestyle changes and medication to help, etc.). Emphasise importance of exercise (e.g., heart health: strengthening heart; overall health: lowering cholesterol; etc.)., Give recommendations for exercise (moderate physical activity: patient’s recovery uncomplicated, already two weeks since hospital admission, etc.). Advise importance of joining cardiac rehabilitation programme (e.g., increasing exercise tolerance under supervision, etc.). Explore patient’s job (type of work, etc.)., Give timescale for return to work (e.g., 4–6 weeks for desk job, etc.). Explain need to be physically and emotionally ready (e.g., not rushing back, planning return to work: assistance/support from employer, etc.). Explore any concerns., Provide recommendations for prevention of future attacks (diet: vegetables, whole grains; lifestyle: physically active; etc.). Reassure patient about his/her concerns (e.g., normal, appropriate, etc.).]
            }"""
    },
    {
        "question": "Generate a card for speaking role play in OET exam. The card must be in JSON format. The card must be in JSON format with 5 keys: SETTINGS; PATIENT_CARD; PATIENT_TASK; DOCTOR_CARD; DOCTOR_TASK. The disease is diabetes",
        "answer": """{
            SETTING: Local Medical Clinic,
            PATIENT_CARD : You are a 45-year-old office worker, and have been feeling tired and unwell. You think you are a bit overweight and are concerned you may have diabetes. Recent publicity about diabetes (on TV, in the newspaper) has made you decide to get a check-up.,
            PATIENT_TASK : [When asked, say lately you’ve been feeling tired and unwell. Sometimes you feel dizzy, thirsty, and breathless; you also have itchy skin., Say you have a busy and stressful office job, and three teenage children, which leaves you no time for exercise. Ask if the symptoms might mean you have diabetes., Say if it is diabetes, you’d like to know how it would be treated., Say that information is helpful but you’re not sure what to do next., Say you’ll do a blood test and make an appointment to discuss the results.],
            DOCTOR_CARD : Your patient is a 45-year-old office worker who is complaining of fatigue and feeling unwell. The patient appears to be overweight and thinks he/she may have diabetes. Recent publicity about diabetes (on TV, in the newspaper) has made him/her decide to get a check-up.
            DOCTOR_TASK : [Find out how patient is feeling (any symptoms, concerns, etc.)., Explore patient’s lifestyle (exercise, work/life balance, etc.)., Discuss possible significance of symptoms (e.g., possible underlying condition such as diabetes, etc.). Reassure patient about his/her symptoms (e.g., different possible causes, diabetes: only one possibility, can be managed, etc.)., Briefly explain diabetes (e.g., type 1: insulin not produced; type 2: insulin not sufficient/effective, etc.). Outline management of diabetes (e.g., medication, diet, exercise, monitoring of blood glucose, etc.)., Outline next steps (diagnostic blood test, return visit for results, consequent assessment of patient’s health and lifestyle, etc.).]
            }"""
    },
    {
        "question": "Generate a card for speaking role play in OET exam. The card must be in JSON format with 5 keys: SETTINGS; PATIENT_CARD; PATIENT_TASK; DOCTOR_CARD; DOCTOR_TASK. The disease is stomach",
        "answer": """{
            SETTING: Emergency Department,
            PATIENT_CARD : You are the parent of a four-year-old boy who came to the Emergency Department two hours ago, after 36 hours of recurrent vomiting and stomach pain. The doctor told you that your son had viral gastroenteritis. He was kept in for two hours on oral re-hydration fluids. Your son is not present for your discussion with the doctor.,
            PATIENT_TASK : [When asked, say you still don’t really understand what viral gastroenteritis is., Say you don’t know how your son got viral gastroenteritis.", "Say your son looks very weak; you really think he needs to be kept in hospital., When asked, say you’re concerned about taking your son home; you just don’t know what to do if he starts to feel worse at home., Say you feel better about taking your son home now that you know what to look for and when to come back to the Emergency Department.],
            DOCTOR_CARD : The parent presented two hours ago at the Emergency Department with his/her four-year-old son. The child had a 36-hour history of recurrent vomiting and stomach pain which was diagnosed as viral gastroenteritis. He was given oral re-hydration fluids and observed for two hours. He is now ready to be discharged. The child is not present for your discussion with the parent.,
            DOCTOR_TASK : [Confirm child is ready to be discharged. Find out about parent’s concerns.", "Explain viral gastroenteritis (e.g., irritation of stomach or intestines due to viral infection, etc.).", "Give information about how gastroenteritis is spread (e.g., contact with vomit/faeces of infected person: shaking hands, contaminated foods/objects, etc.).", "Resist request to keep child in hospital (e.g., oral re-hydration therapy usually effective, etc.). Advise on hydration and appropriate clear fluids (e.g., watered down unsweetened fruit juice, electrolyte drinks, etc.). Find out any other concerns.", "Advise when to seek medical advice (e.g., signs of severe dehydration: extreme thirst, lethargy, irritability, pale/sunken eyes, etc.).]
        }"""
    }
]

# with open('../data/oet_cards_examples.json', 'w') as file:
#     json.dump(oet_cards, file)

In [117]:
import random


few_shot_content = """
You are a medicine spiking card for the OET exam creating AI tool. Don't so anything except cards creation.
You have to come up with a SETTING where the event take place.
You make cards for general practitioners, so the illnesses should be appropriate. 
Remember that this exam is to test English, so do not use too complicated words, write in a way that a student with English level B1 can understand.
Think of a SITUATION with which the patient came and 5 TASKS each for the PATIENT and the DOCTOR. Which includes a description of the situation, complaints and questions of a patient, and for the doctor, who should find out what the problem is, explain the causes and symptoms, make a diagnosis and prescribe treatment. 
Don't forget to write for the doctor's recommendations of what needs to be advised or clarified. A list of things that may have caused the symptoms or illness that the doctor can clarify or a list of things that could be advised. These are listed in parentheses after the description of one item in the assignment. Act on the situation, don't list it everywhere, but where relevant, write it down.
"""

def search_for_disease():
    '''Search for examples with exact disease
    '''
    return [card for card in oet_cards if disease in card['question'].lower()]

def make_examples(examples, disease = 'not stated', num_examples=2):
    if disease != 'not stated':
        examples = search_for_disease(disease)
    prompt_examples = random.sample(examples, num_examples)
    return prompt_examples
    
def make_prompt(content, examples, disease = 'not stated', num_examples=2):
    messages = [
        {"role": "system", "content": content}
    ]

    prompt_examples = make_examples(examples, disease, num_examples)
    
    for ex in prompt_examples:
        messages.append({"role": "user", "content": ex['question']})
        messages.append({"role": "assistant", "content": ex['answer']})

    messages.append({"role": "user", "content": f"Generate a card for speaking role play in OET exam. The card must be in JSON format with 5 keys: SETTINGS; PATIENT_CARD; PATIENT_TASK; DOCTOR_CARD; DOCTOR_TASK. The disease is {disease}"})
    
    return messages
    
res = make_prompt(few_shot_content, oet_cards, num_examples=2)

In [118]:
res

[{'role': 'system',
  'content': "\nYou are a medicine spiking card for the OET exam creating AI tool. Don't so anything except cards creation.\nYou have to come up with a SETTING where the event take place.\nYou make cards for general practitioners, so the illnesses should be appropriate. \nRemember that this exam is to test English, so do not use too complicated words, write in a way that a student with English level B1 can understand.\nThink of a SITUATION with which the patient came and 5 TASKS each for the PATIENT and the DOCTOR. Which includes a description of the situation, complaints and questions of a patient, and for the doctor, who should find out what the problem is, explain the causes and symptoms, make a diagnosis and prescribe treatment. \nDon't forget to write for the doctor's recommendations of what needs to be advised or clarified. A list of things that may have caused the symptoms or illness that the doctor can clarify or a list of things that could be advised. These

In [119]:
def few_shot_card_generation(client, messages):
    completion = client.chat.completions.create(
        model=openai_model,
        response_format={"type": "json_object"},
        messages=messages,
        max_tokens=1000
    )

    return completion.choices[0].message.content 

oet_client = OpenAI(
    api_key=os.getenv('OPENAI_API_TOKEN'),
    organization=os.getenv('OPENAI_ORG_ID'), # unnecessary
    project=os.getenv('OPENAI_PROJ_OET'), # to track expenses
)

gen_card = few_shot_card_generation(oet_client, res)

In [123]:
card_json = json.loads(gen_card)
card_json

{'SETTING': 'Geriatric Care Facility',
 'PATIENT_CARD': 'You are an 85-year-old woman who recently moved into a geriatric care facility. You have been experiencing frequent urination, especially at night, and sometimes feel a burning sensation when passing urine. You are concerned about these symptoms and are requesting a medical check-up.',
 'PATIENT_TASK': ['Express your concerns about the frequent urination and burning sensation when passing urine.',
  'Mention that you have also noticed a slight decrease in your appetite and energy levels recently.',
  'Ask the doctor if these symptoms could be related to your age or if they indicate a specific medical condition.',
  'Inquire about the possible treatment options for your symptoms.',
  'Request advice on how to manage these symptoms effectively.'],
 'DOCTOR_CARD': 'Your patient is an 85-year-old woman who recently moved into a geriatric care facility. She is experiencing frequent urination, especially at night, and a burning sensati

In [126]:
card_json.keys()

dict_keys(['SETTING', 'PATIENT_CARD', 'PATIENT_TASK', 'DOCTOR_CARD', 'DOCTOR_TASK'])

### Make a print as message

In [135]:
assert list(card_json.keys()) == ['SETTING', 'PATIENT_CARD', 'PATIENT_TASK', 'DOCTOR_CARD', 'DOCTOR_TASK']
assert len(card_json['PATIENT_TASK']) == 5 and len(card_json['DOCTOR_TASK']) == 5

In [151]:
from io import StringIO


def generate_oet_card_text(card):
    card_data = eval(card)  # Convert the string representation of the dictionary back to a dictionary
    buffer = StringIO()
    
    buffer.write("# OET Exam Role Play Card\n\n")
    buffer.write(f"## Setting: {card_data['SETTING']}\n\n")
    buffer.write(f"## Patient Card\n{card_data['PATIENT_CARD']}\n\n")
    buffer.write("## Patient Tasks\n")
    for i, task in enumerate(card_data['PATIENT_TASK'], 1):
        buffer.write(f"{i}. {task}\n")
        
    buffer.write(f"\n## Doctor Card\n{card_data['DOCTOR_CARD']}\n\n")
    buffer.write("## Doctor Tasks\n")
    for i, task in enumerate(card_data['DOCTOR_TASK'], 1):
        buffer.write(f"{i}. {task}\n")
    
    buffer.seek(0)
    return buffer

buffer = generate_oet_card_text(gen_card)
text = buffer.getvalue()
print(text)

# OET Exam Role Play Card

## Setting: Geriatric Care Facility

## Patient Card
You are an 85-year-old woman who recently moved into a geriatric care facility. You have been experiencing frequent urination, especially at night, and sometimes feel a burning sensation when passing urine. You are concerned about these symptoms and are requesting a medical check-up.

## Patient Tasks
1. Express your concerns about the frequent urination and burning sensation when passing urine.
2. Mention that you have also noticed a slight decrease in your appetite and energy levels recently.
3. Ask the doctor if these symptoms could be related to your age or if they indicate a specific medical condition.
4. Inquire about the possible treatment options for your symptoms.
5. Request advice on how to manage these symptoms effectively.

## Doctor Card
Your patient is an 85-year-old woman who recently moved into a geriatric care facility. She is experiencing frequent urination, especially at night, and a burn

In [166]:
import textwrap
from fpdf import FPDF
from PIL import Image, ImageDraw, ImageFont


def create_pdf_from_buffer(buffer, filename="../data/card_output.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=0)
    pdf.set_font("Arial", size=10)
    
    # Split the text into lines and add each line to the PDF
    text = buffer.getvalue()
    for line in text.split('\n'):
        pdf.multi_cell(0, 10, txt=line, align='L')
    
    pdf.output(filename)

def create_jpeg_from_buffer(buffer, filename="../data/card_output.jpg"):
    # Create an image with white background
    img = Image.new('RGB', (700, 800), color=(255, 255, 255))
    d = ImageDraw.Draw(img)
    
    # Use a truetype font
    try:
        font = ImageFont.truetype("arial.ttf", 15)
    except IOError:
        font = ImageFont.load_default()
    
    # Split the text into lines and draw each line on the image
    text = buffer.getvalue()
    margin = 40
    offset = 40
    for line in text.split('\n'):
        wrapped_text = textwrap.fill(line, width=85)  # Adjust width as needed
        for wrap_line in wrapped_text.split('\n'):
            d.text((margin, offset), wrap_line, font=font, fill=(0, 0, 0))
            offset += 15  # Move down for next line
            if offset > img.size[1] - 40:  # Add pages if needed
                new_img = Image.new('RGB', (700, 800), color=(255, 255, 255))
                new_img.paste(img, (0, 0))
                img = new_img
                d = ImageDraw.Draw(img)
                offset = 40
    
    img.save(filename)

create_pdf_from_buffer(buffer)
create_jpeg_from_buffer(buffer)

# 2. Show the Card and wait 3 minutes.
* At this point we have to show the user his card and let him prepare with it for 3 minutes

In [10]:
import time


print('You know have 3 minutes to prepare', formatted_cards['doctor'], sep='\n')
time.sleep(1) ## 180

You know have 3 minutes to prepare
Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily activities despite trying over-the-counter creams.
**DOCTOR TASKS**
  - Inquire about the duration and progression of the symptoms.
  - Discuss potential triggers for the itching and redness (e.g., new products, environmental changes).
  - Explain the possible causes of the symptoms (e.g., contact dermatitis, eczema) and the importance of a proper diagnosis.
  - Recommend allergy testing or patch testing to identify potential allergens.
  - Provide treatment options such as prescription creams, lifestyle modifications, and follow-up care.



# 3. Generate a Voice Message.
* Voice Message from the User. It will be just sent to us in telegram, but here I need to generate it.

## 3.1 Google Text-to-Speech 
* I liked this the most

In [11]:
# !pip install gtts -q

In [15]:
from gtts import gTTS

user = 'Pete'
text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

tts = gTTS(text=text, lang='en')

path = f"../data/audio_storage/{user}_intro_gtts.mp3"
tts.save(path)

os.system(f"start {path}")  # For Windows

0

In [16]:
def create_audio(text, path):
    tts = gTTS(text=text, lang='en')
    tts.save(path)

## 3.2 pyttsx3 is a text-to-speech conversion library in Python

* Didn't like the sound.

In [17]:
# import pyttsx3

# # Initialize the TTS engine
# engine = pyttsx3.init()

# # Text to convert to speech
# text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

# # Set properties (optional)
# engine.setProperty('rate', 200)  # Speed of speech
# engine.setProperty('volume', 0.9)  # Volume (0.0 to 1.0)

# # Convert text to speech
# path = f"data/audio_storage/{user}_intro_pyttsx3.mp3"
# engine.save_to_file(text, path)
# engine.runAndWait()

# os.system(f"start {path}")  # For Windows

## 3.3 TTS

In [18]:
# import torch
# from TTS.api import TTS

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(device)

# # print(TTS().list_models().list_tts_models())

# tts = TTS('tts_models/en/ljspeech/speedy-speech').to(device)

# text = f"Hello, my name is {user}, I am a doctor of this clinic. May I know your name and please tell me what is bothering you?"

# path = f"data/audio_storage/{user}_intro_tts.mp3"
# tts.tts_to_file(text=text, file_path=path) # speaker_wav="my/cloning/audio.wav"

# os.system(f"start {path}") # For Windows

# 4. Transcribing the Voice Message with Whisper

In [19]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [20]:
# !pip install git+https://github.com/openai/whisper.git

In [21]:
import torch
import torchaudio
from torch.cuda.amp import autocast
import whisper

device = "cuda" if torch.cuda.is_available() else "CPU"
print(device)

# Select from the following models: "tiny", "base", "small", "medium", "large"
whisper_model = whisper.load_model("base", device=device)

cuda


In [22]:
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    return waveform, sample_rate

def create_audio_chunks(waveform, total_duration, sample_rate, chunk_duration=30):
    chunks = []
    for start in range(0, int(total_duration), chunk_duration):
        end = min(start + chunk_duration, total_duration)
        chunk_waveform = waveform[:, int(start * sample_rate):int(end * sample_rate)]
        chunks.append(chunk_waveform)
    return chunks, sample_rate

def transcribe_with_whisper(model, audio_path):
    if not os.path.isfile(audio_path):
        raise FileNotFoundError(f"The file {audio_path} does not exist.")
    else: 
        waveform, sample_rate = load_audio(audio_path)
        total_duration = waveform.size(1) / sample_rate
        
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available. Please check your CUDA installation.")

    if total_duration <= 30:
        waveform = waveform.to(device)

        # Whisper expects audio to be resampled to 16000 Hz
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000).to(device)
        waveform = resampler(waveform)

        audio_tensor = waveform.squeeze(0).cpu().numpy()  # Whisper expects a 1D numpy array
        options = {"fp16": False, "language": None, "task": "transcribe"}
        result = model.transcribe(audio_tensor, **options)
        
        return result['text']
    else:
        chunks, sample_rate = create_audio_chunks(waveform, total_duration, sample_rate)
    
        # Whisper expects audio to be resampled to 16000 Hz
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000).to(device)
        transcriptions = []
    
        for chunk in chunks:
            chunk = chunk.to(device)
            chunk = resampler(chunk)
            
            with autocast():
                audio_tensor = chunk.squeeze(0).cpu().numpy()  # Whisper expects a 1D numpy array
                result = model.transcribe(audio_tensor)
                transcriptions.append(result['text'])
    
            torch.cuda.empty_cache() # Release GPU memory
    
        return " ".join(transcriptions)

## 4.1 Small text transcribing

In [23]:
%%time

transcribed_text = transcribe_with_whisper(whisper_model, path)
transcribed_text

CPU times: total: 3.22 s
Wall time: 1.69 s


' Hello, my name is Pete. I am a doctor of this clinic. May I know your name and please tell me what is bothering you?'

## 4.2 Big text transcribing

In [24]:
long_text = '''
In a bustling town, there lived a curious kitten named Milo. Milo had bright green eyes and a fluffy tail that always seemed to be in motion. He loved to explore every nook and cranny, making new friends along the way. One day, while chasing a butterfly, Milo wandered farther than ever before and stumbled upon an old, enchanted garden.
In the garden, flowers sang melodies, and trees whispered ancient secrets. Milo's whiskers twitched with excitement. As he ventured deeper, he found a tiny, golden door at the base of a giant oak tree. Unable to resist, Milo pushed it open and stepped inside.
To his amazement, he found himself in a land of miniature creatures. Tiny fairies flitted about, and little gnomes tended to their mushroom houses. The ruler of this land, Queen Lila, approached Milo with a warm smile. "Welcome, Milo. We need your help," she said.
Queen Lila explained that a mischievous squirrel named Nibbles had stolen their magical acorn, which kept the garden alive. Without it, the garden would wither away. Brave and determined, Milo agreed to help.
With the fairies' guidance, Milo tracked down Nibbles and persuaded him to return the acorn. In gratitude, Queen Lila gifted Milo a magical collar that would always guide him home. Milo returned to his town, carrying memories of a fantastical adventure and a heart full of new friendships. From then on, Milo knew that no matter where his curiosity led him, he would always find his way back home.
'''

In [36]:
%%time

user = 'Pete'
long_audio_path = f"../data/audio_storage/{user}_long_gtts.mp3"
create_audio(long_text, long_audio_path)

CPU times: total: 1.09 s
Wall time: 8.85 s


In [37]:
%%time

transcribed_text = transcribe_with_whisper(whisper_model, long_audio_path)
transcribed_text

CPU times: total: 12.2 s
Wall time: 6.36 s


' in a bustling town. Their lived a curious kitten named Milo. Milo had bright green eyes and a fluffy tail that always seemed to be in motion. He loved to explore every nook and cranny, making new friends along the way. One day, while chasing a butterfly, Milo wandered farther than ever before and stumbled upon an old, enchanted garden. In the garden, flowers sang melodies.  and Trees whispered ancient secrets. Milo\'s whiskers twitched with excitement. As he ventured deeper, he found a tiny, golden door at the base of a giant oak tree, unable to resist. Milo pushed it open and stepped inside. To his amazement, he found himself in a land of miniature creatures. Tiny fairies flittered about. And little gnomes tended to their mushroom houses. The ruler...  of this land. Queen Lila approached Milo with a warm smile. Welcome! Milo! We need your help!" she said. Queen Lila explained that a mischievous squirrel named nibbles had stolen their magical acorn. Which kept the garden alive. Witho

# 4* Assessing the accuracy of text transcription

In [38]:
import re
import Levenshtein
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import ndiff

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def calculate_levenshtein_distance(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)
    distance = Levenshtein.distance(original_text_norm, transcribed_text_norm)
    return distance

def calculate_cosine_similarity(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)

    original_tokens = word_tokenize(original_text_norm)
    transcribed_tokens = word_tokenize(transcribed_text_norm)

    original_text_combined = ' '.join(original_tokens)
    transcribed_text_combined = ' '.join(transcribed_tokens)

    vectorizer = TfidfVectorizer().fit_transform([original_text_combined, transcribed_text_combined])
    vectors = vectorizer.toarray()

    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0, 1]

def detailed_diff(original_text, transcribed_text):
    original_text_norm = normalize_text(original_text)
    transcribed_text_norm = normalize_text(transcribed_text)
    diff = ndiff(original_text_norm.split(), transcribed_text_norm.split())
    return '\n'.join(diff)

In [39]:
levenshtein_distance = calculate_levenshtein_distance(long_text, transcribed_text)
print(f"Levenshtein Distance: {levenshtein_distance}")

cosine_sim = calculate_cosine_similarity(long_text, transcribed_text)
print(f"Cosine Similarity: {cosine_sim:.2f}")

diff = detailed_diff(long_text, transcribed_text)
print(" ", "Detailed Differences:", diff, sep='\n')

Levenshtein Distance: 17
Cosine Similarity: 0.99
 
Detailed Differences:
  in
  a
  bustling
  town
- there
?     -

+ their
?    +

  lived
  a
  curious
  kitten
  named
  milo
  milo
  had
  bright
  green
  eyes
  and
  a
  fluffy
  tail
  that
  always
  seemed
  to
  be
  in
  motion
  he
  loved
  to
  explore
  every
  nook
  and
  cranny
  making
  new
  friends
  along
  the
  way
  one
  day
  while
  chasing
  a
  butterfly
  milo
  wandered
  farther
  than
  ever
  before
  and
  stumbled
  upon
  an
  old
  enchanted
  garden
  in
  the
  garden
  flowers
  sang
  melodies
  and
  trees
  whispered
  ancient
  secrets
  milos
  whiskers
  twitched
  with
  excitement
  as
  he
  ventured
  deeper
  he
  found
  a
  tiny
  golden
  door
  at
  the
  base
  of
  a
  giant
  oak
  tree
  unable
  to
  resist
  milo
  pushed
  it
  open
  and
  stepped
  inside
  to
  his
  amazement
  he
  found
  himself
  in
  a
  land
  of
  miniature
  creatures
  tiny
  fairies
- flitt

# Role Play

## Disription
* **Step 1: Creating and Sending the Card to the User** <br> Let's start by creating a card and sending it to the user.
* **Step 2: Let the user familiarize himself with his card**
* **Step 3: Starting a 5-Minute Timer for Role-Play** <br> Start a 5-minute timer and begin accepting a voice message from the user.
* **Loop:**
    * **Step 3.1: Receiving a Voice Message from the User** <br> Use the python-telegram-bot library to handle voice messages.
    * **Step 3.2: Transcribing the Voice Message with Whisper** <br> Integrate Whisper for processing voice messages.
    * **Step 3.3: Generating an Audio Response - Patient's Reply** <br> Use OpenAI GPT-4 to generate the patient's reply text.
    * **Step 3.4: Sending the Audio Response to the User** <br> Use Google Text-to-Speech to create an audio file and send the generated audio file to the user.

In [40]:
print('Now introduce yourself as a doctor and record your first voice message. You have 5 minutes.')

# 300 seconds timer

Now introduce yourself as a doctor and record your first voice message. You have 5 minutes.


We've received the first audio from a user

In [41]:
user_text = transcribe_with_whisper(whisper_model, path)
user_text

' Hello, my name is Pete. I am a doctor of this clinic. May I know your name and please tell me what is bothering you?'

Patient model content.

In [42]:
system_content = f"""
You play a patient in a role-playing game, you have a situation card: { {formatted_cards['patient']} }.
All you need to do is to FOLLOW the role play card.
Don't dump all the information at once!
You may be asked to tell what you came with, don't answer with sentences from the task, just give a short description (take it from your card).
Your task is to complete your tasks one by one, either until you are asked about something or until you are asked what is bothering you. 
Try not to go beyond what's on your card. If you are asked something not on your card, answer it briefly.
"""
print(system_content)

# Your name is AI patient.
# If they ask you your name - you can make it up, but regarding the doctor's questions about your condition, answer only what is on the notes.
# Do not move on to the next question until you have received an answer to the previous one. 


You play a patient in a role-playing game, you have a situation card: {"Patient Card:\n**SETTING**: Dermatology Clinic\n**PATIENT**: You are a 30-year-old woman who has been experiencing severe itching and redness on your skin for the past few weeks. You have noticed the symptoms mainly on your arms, legs, and back. The itching is disrupting your sleep and daily activities, and you have tried over-the-counter creams with no relief.\n**PATIENT TASKS**\n  - Describe the severity of the itching and redness on your skin, mentioning the areas affected.\n  - Express how the itching is impacting your sleep and daily life.\n  - Inform the doctor about the unsuccessful use of over-the-counter creams.\n  - Ask the doctor about the possible causes of your symptoms and if it could be a skin allergy.\n  - Acknowledge the doctor's recommendations and ask about the next steps.\n"}.
All you need to do is to FOLLOW the role play card.
Don't dump all the information at once!
You may be asked to tell wh

In [43]:
messages = [{"role": "system", "content": system_content}] # Append a system content

def chatbot_response(user_input):
    messages.append({"role": "user", "content": user_input}) # Append user's message
    
    completion = client.chat.completions.create(
        model=openai_model,
        messages=messages,
        max_tokens=200
    )
    
    output_text = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": output_text}) # Append assistant's message
    return output_text

In [45]:
print(formatted_cards['doctor'])

Doctor Card:
**SETTING**: Dermatology Clinic
**DOCTOR**: Your patient is a 30-year-old woman presenting with severe itching and redness on her arms, legs, and back. She reports disrupted sleep and daily activities despite trying over-the-counter creams.
**DOCTOR TASKS**
  - Inquire about the duration and progression of the symptoms.
  - Discuss potential triggers for the itching and redness (e.g., new products, environmental changes).
  - Explain the possible causes of the symptoms (e.g., contact dermatitis, eczema) and the importance of a proper diagnosis.
  - Recommend allergy testing or patch testing to identify potential allergens.
  - Provide treatment options such as prescription creams, lifestyle modifications, and follow-up care.



In [44]:
user_input = input("You: ")

while user_input.lower() != 'exit':
    response = chatbot_response(user_input)
    print(f"AI: {response}")
    
    user_input = input("You: ")

You:  exit
